# Context Engineering 연구 노트북

DeepAgents 라이브러리에서 사용되는 5가지 Context Engineering 전략을 분석하고 실험합니다.

## 참고 자료

- YouTube: https://www.youtube.com/watch?v=6_BcCthVvb8
- PDF: Context Engineering Meetup.pdf
- PDF: Manus Context Engineering LangChain Webinar.pdf

## 문제 정의(왜 필요한가)

- 에이전트는 도구 호출(tool calls)과 관찰(observations)이 누적되며 컨텍스트가 계속 성장합니다(수십~수백 턴).
- 컨텍스트가 길어질수록 성능이 떨어질 수 있다는 관측이 있습니다(context rot).
- 실패 모드: Poisoning / Distraction / Confusion / Clash

## Manus 관점(모델 vs 앱 경계)

- Context Engineering은 application과 model 사이의 실용적인 경계로 다뤄집니다.
- ‘모델을 따로 학습/미세조정’에 먼저 뛰어들기보다, 컨텍스트 설계로 제품 반복 속도를 확보한다는 관점이 강조됩니다.

## 추가 주제: Tool Offloading

- 도구 자체도 컨텍스트를 더럽힐 수 있으므로, 계층적 액션 스페이스/도구 로딩 제한(필요한 도구만 노출)을 고려합니다.

## Context Engineering 5가지 핵심 전략

| 전략 | 설명 | DeepAgents 구현 |
|------|------|----------------|
| **1. Offloading** | 대용량 결과를 파일로 축출 | FilesystemMiddleware |
| **2. Reduction** | Compaction + Summarization | SummarizationMiddleware |
| **3. Retrieval** | grep/glob 기반 검색 | FilesystemMiddleware |
| **4. Isolation** | SubAgent로 컨텍스트 격리 | SubAgentMiddleware |
| **5. Caching** | Prompt Caching | AnthropicPromptCachingMiddleware |

## 아키텍처 개요

```
┌─────────────────────────────────────────────────────────────────┐
│                     Context Engineering                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌────────────┐    ┌────────────┐    ┌────────────┐            │
│   │ Offloading │    │ Reduction  │    │  Caching   │            │
│   │ (20k 토큰) │    │ (85% 임계) │    │ (Anthropic)│            │
│   └─────┬──────┘    └─────┬──────┘    └─────┬──────┘            │
│         │                 │                 │                    │
│         ▼                 ▼                 ▼                    │
│   ┌─────────────────────────────────────────────────────┐       │
│   │              Middleware Stack                       │       │
│   └─────────────────────────────────────────────────────┘       │
│                          │                                       │
│         ┌────────────────┼────────────────┐                     │
│         ▼                ▼                ▼                     │
│   ┌────────────┐  ┌────────────┐  ┌────────────┐               │
│   │ Retrieval  │  │ Isolation  │  │  Backend   │               │
│   │(grep/glob) │  │ (SubAgent) │  │ (FileSystem│               │
│   └────────────┘  └────────────┘  └────────────┘               │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
import sys
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(".env", override=True)

PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

---

## 전략 1: Context Offloading

대용량 도구 결과를 파일시스템으로 축출하여 컨텍스트 윈도우 오버플로우를 방지합니다.

### 핵심 원리
- 도구 결과가 `tool_token_limit_before_evict` (기본 20,000 토큰) 초과 시 자동 축출
- `/large_tool_results/{tool_call_id}` 경로에 저장
- 처음 10줄 미리보기 제공
- 에이전트가 `read_file`로 필요할 때 로드

In [1]:
from context_engineering_research_agent.context_strategies.offloading import (
    ContextOffloadingStrategy,
    OffloadingConfig,
)

config = OffloadingConfig(
    token_limit_before_evict=20000,
    eviction_path_prefix="/large_tool_results",
    preview_lines=10,
)

print(f"토큰 임계값: {config.token_limit_before_evict:,}")
print(f"축출 경로: {config.eviction_path_prefix}")
print(f"미리보기 줄 수: {config.preview_lines}")

토큰 임계값: 20,000
축출 경로: /large_tool_results
미리보기 줄 수: 10


In [2]:
strategy = ContextOffloadingStrategy(config=config)

small_content = "짧은 텍스트" * 100
large_content = "대용량 텍스트" * 30000

print(f"짧은 콘텐츠: {len(small_content)} 자 → 축출 대상: {strategy._should_offload(small_content)}")
print(f"대용량 콘텐츠: {len(large_content):,} 자 → 축출 대상: {strategy._should_offload(large_content)}")

짧은 콘텐츠: 600 자 → 축출 대상: False
대용량 콘텐츠: 210,000 자 → 축출 대상: True


---

## 전략 2: Context Reduction

컨텍스트 윈도우 사용량이 임계값을 초과할 때 자동으로 대화 내용을 압축합니다.

### 두 가지 기법

| 기법 | 설명 | 비용 |
|------|------|------|
| **Compaction** | 오래된 도구 호출/결과 제거 | 무료 |
| **Summarization** | LLM이 대화 요약 | API 비용 발생 |

우선순위: Compaction → Summarization

In [3]:
from context_engineering_research_agent.context_strategies.reduction import (
    ContextReductionStrategy,
    ReductionConfig,
)

config = ReductionConfig(
    context_threshold=0.85,
    model_context_window=200000,
    compaction_age_threshold=10,
    min_messages_to_keep=5,
)

print(f"임계값: {config.context_threshold * 100}%")
print(f"컨텍스트 윈도우: {config.model_context_window:,} 토큰")
print(f"Compaction 대상 나이: {config.compaction_age_threshold} 메시지")
print(f"최소 유지 메시지: {config.min_messages_to_keep}")

임계값: 85.0%
컨텍스트 윈도우: 200,000 토큰
Compaction 대상 나이: 10 메시지
최소 유지 메시지: 5


In [4]:
from langchain_core.messages import AIMessage, HumanMessage

strategy = ContextReductionStrategy(config=config)

messages = [
    HumanMessage(content="안녕하세요" * 1000),
    AIMessage(content="안녕하세요" * 1000),
] * 20

usage_ratio = strategy._get_context_usage_ratio(messages)
print(f"컨텍스트 사용률: {usage_ratio * 100:.1f}%")
print(f"축소 필요: {strategy._should_reduce(messages)}")

컨텍스트 사용률: 25.0%
축소 필요: False


---

## 전략 3: Context Retrieval

grep/glob 기반의 단순하고 빠른 검색으로 필요한 정보만 선택적으로 로드합니다.

### 벡터 검색을 사용하지 않는 이유

| 특성 | 직접 검색 | 벡터 검색 |
|------|----------|----------|
| 결정성 | ✅ 정확한 매칭 | ❌ 확률적 |
| 인프라 | ✅ 불필요 | ❌ 벡터 DB 필요 |
| 속도 | ✅ 빠름 | ❌ 인덱싱 오버헤드 |
| 디버깅 | ✅ 예측 가능 | ❌ 블랙박스 |

In [5]:
from context_engineering_research_agent.context_strategies.retrieval import (
    ContextRetrievalStrategy,
    RetrievalConfig,
)

config = RetrievalConfig(
    default_read_limit=500,
    max_grep_results=100,
    max_glob_results=100,
    truncate_line_length=2000,
)

print(f"기본 읽기 제한: {config.default_read_limit} 줄")
print(f"grep 최대 결과: {config.max_grep_results}")
print(f"glob 최대 결과: {config.max_glob_results}")
print(f"줄 길이 제한: {config.truncate_line_length} 자")

기본 읽기 제한: 500 줄
grep 최대 결과: 100
glob 최대 결과: 100
줄 길이 제한: 2000 자


---

## 전략 4: Context Isolation

SubAgent를 통해 독립된 컨텍스트 윈도우에서 작업을 수행합니다.

### 장점
- 메인 에이전트 컨텍스트 오염 방지
- 복잡한 작업의 격리 처리
- 병렬 처리 가능

### SubAgent 유형

| 유형 | 구조 | 특징 |
|------|------|------|
| Simple | `{name, system_prompt, tools}` | 단일 응답 |
| Compiled | `{name, runnable}` | 자체 DeepAgent, 다중 턴 |

In [6]:
from context_engineering_research_agent.context_strategies.isolation import (
    ContextIsolationStrategy,
    IsolationConfig,
)

config = IsolationConfig(
    default_model="gpt-4.1",
    include_general_purpose_agent=True,
    excluded_state_keys=("messages", "todos", "structured_response"),
)

print(f"기본 모델: {config.default_model}")
print(f"범용 에이전트 포함: {config.include_general_purpose_agent}")
print(f"제외 상태 키: {config.excluded_state_keys}")

기본 모델: gpt-4.1
범용 에이전트 포함: True
제외 상태 키: ('messages', 'todos', 'structured_response')


---

## 전략 5: Context Caching

Anthropic Prompt Caching을 활용하여 시스템 프롬프트와 반복 컨텍스트를 캐싱합니다.

### 이점
- API 호출 비용 절감
- 응답 속도 향상
- 동일 세션 내 반복 호출 최적화

### 캐싱 조건
- 최소 1,024 토큰 이상
- `cache_control: {"type": "ephemeral"}` 마커 추가

In [8]:
from context_engineering_research_agent.context_strategies.caching import (
    ContextCachingStrategy,
    CachingConfig,
)

config = CachingConfig(
    min_cacheable_tokens=1024,
    cache_control_type="ephemeral",
    enable_for_system_prompt=True,
    enable_for_tools=True,
)

print(f"최소 캐싱 토큰: {config.min_cacheable_tokens:,}")
print(f"캐시 컨트롤 타입: {config.cache_control_type}")
print(f"시스템 프롬프트 캐싱: {config.enable_for_system_prompt}")
print(f"도구 캐싱: {config.enable_for_tools}")

최소 캐싱 토큰: 1,024
캐시 컨트롤 타입: ephemeral
시스템 프롬프트 캐싱: True
도구 캐싱: True


In [9]:
strategy = ContextCachingStrategy(config=config)

short_content = "짧은 시스템 프롬프트"
long_content = "긴 시스템 프롬프트 " * 500

print(f"짧은 콘텐츠: {len(short_content)} 자 → 캐싱 대상: {strategy._should_cache(short_content)}")
print(f"긴 콘텐츠: {len(long_content):,} 자 → 캐싱 대상: {strategy._should_cache(long_content)}")

짧은 콘텐츠: 11 자 → 캐싱 대상: False
긴 콘텐츠: 5,500 자 → 캐싱 대상: True


---

## 통합 에이전트 실행

5가지 전략이 모두 적용된 에이전트를 실행합니다.

In [11]:
from context_engineering_research_agent import create_context_aware_agent

agent = create_context_aware_agent(
    model="gpt-4.1",
    enable_offloading=True,
    enable_reduction=True,
    enable_caching=True,
    offloading_token_limit=20000,
    reduction_threshold=0.85,
)

print(f"에이전트 타입: {type(agent).__name__}")

에이전트 타입: CompiledStateGraph


---

## 전략 활성화/비활성화 비교 실험

각 전략을 활성화/비활성화했을 때의 차이점을 실험합니다.

### 실험 설계

| 실험 | Offloading | Reduction | Caching | 목적 |
|------|------------|-----------|---------|------|
| 1. 기본 | ❌ | ❌ | ❌ | 베이스라인 |
| 2. Offloading만 | ✅ | ❌ | ❌ | 대용량 결과 축출 효과 |
| 3. Reduction만 | ❌ | ✅ | ❌ | 컨텍스트 압축 효과 |
| 4. 모두 활성화 | ✅ | ✅ | ✅ | 전체 효과 |

### 실패 모드(컨텍스트가 커질 때) 시뮬레이션 실험

아래 실험 5~8은 **API 키 없이 실행 가능한 순수 파이썬 시뮬레이션**으로, “컨텍스트 실패 모드”를 재현하고 완화책을 보여줍니다.

| 실험 | 실패 모드 | 무엇을 재현하나 | 완화책(예시) |
|------|----------|-----------------|--------------|
| 5 | Confusion | 도구가 많고 유사할수록 선택이 흔들림 | 도구 로딩 제한 / 계층적 액션 스페이스 |
| 6 | Clash | 연속된 관찰이 서로 모순될 때 혼란 | 충돌 감지 / 재검증 / 불확실성 표기 |
| 7 | Distraction | 긴 로그에서 반복 행동으로 쏠림 | 계획/목표 리프레시 / 강제 다음 행동 |
| 8 | Poisoning | 검증되지 않은 사실이 메모리를 오염 | 출처 태깅 / 검증 게이트 / 격리 |


### 실험 1: Offloading 전략 효과

대용량 도구 결과가 있을 때 Offloading 활성화/비활성화 비교

In [13]:
small_result = "검색 결과: 항목 1, 항목 2, 항목 3"
large_result = "\n".join([f"검색 결과 {i}: " + "상세 내용 " * 100 for i in range(500)])

print(f"작은 결과 크기: {len(small_result):,} 자")
print(f"대용량 결과 크기: {len(large_result):,} 자")
print()

offloading_disabled = ContextOffloadingStrategy(
    config=OffloadingConfig(token_limit_before_evict=999999999)
)
offloading_enabled = ContextOffloadingStrategy(
    config=OffloadingConfig(token_limit_before_evict=20000)
)

print("[Offloading 비활성화 시]")
print(f"  작은 결과 축출: {offloading_disabled._should_offload(small_result)}")
print(f"  대용량 결과 축출: {offloading_disabled._should_offload(large_result)}")
print(f"  → 대용량 결과가 컨텍스트에 그대로 포함됨")
print()

print("[Offloading 활성화 시]")
print(f"  작은 결과 축출: {offloading_enabled._should_offload(small_result)}")
print(f"  대용량 결과 축출: {offloading_enabled._should_offload(large_result)}")
print(f"  → 대용량 결과는 파일로 저장, 미리보기만 컨텍스트에 포함")
print()

preview = offloading_enabled._create_preview(large_result)
print(f"미리보기 크기: {len(preview):,} 자 (원본의 {len(preview)/len(large_result)*100:.1f}%)")

작은 결과 크기: 23 자
대용량 결과 크기: 305,889 자

[Offloading 비활성화 시]
  작은 결과 축출: False
  대용량 결과 축출: False
  → 대용량 결과가 컨텍스트에 그대로 포함됨

[Offloading 활성화 시]
  작은 결과 축출: False
  대용량 결과 축출: True
  → 대용량 결과는 파일로 저장, 미리보기만 컨텍스트에 포함

미리보기 크기: 6,159 자 (원본의 2.0%)


### 실험 2: Reduction 전략 효과

긴 대화에서 Compaction 적용 전/후 비교

In [14]:
messages_with_tools = []
for i in range(30):
    messages_with_tools.append(HumanMessage(content=f"질문 {i}: " + "내용 " * 50))
    ai_msg = AIMessage(
        content=f"답변 {i}: " + "응답 " * 50,
        tool_calls=[{'id': f'call_{i}', 'name': 'search', 'args': {'q': 'test'}}] if i < 25 else []
    )
    messages_with_tools.append(ai_msg)
    if i < 25:
        messages_with_tools.append(ToolMessage(content=f"도구 결과 {i}: " + "결과 " * 30, tool_call_id=f'call_{i}'))

reduction = ContextReductionStrategy(
    config=ReductionConfig(compaction_age_threshold=10)
)

original_tokens = reduction._estimate_tokens(messages_with_tools)
print(f"[Reduction 비활성화 시]")
print(f"  메시지 수: {len(messages_with_tools)}")
print(f"  추정 토큰: {original_tokens:,}")
print(f"  → 모든 도구 호출/결과가 컨텍스트에 유지됨")
print()

compacted, result = reduction.apply_compaction(messages_with_tools)
compacted_tokens = reduction._estimate_tokens(compacted)

print(f"[Reduction 활성화 시 - Compaction]")
print(f"  메시지 수: {len(messages_with_tools)} → {len(compacted)}")
print(f"  추정 토큰: {original_tokens:,} → {compacted_tokens:,}")
print(f"  절약된 토큰: {result.estimated_tokens_saved:,} ({result.estimated_tokens_saved/original_tokens*100:.1f}%)")
print(f"  → 오래된 도구 호출/결과가 제거되어 컨텍스트 효율화")

[Reduction 비활성화 시]
  메시지 수: 85
  추정 토큰: 2,972
  → 모든 도구 호출/결과가 컨텍스트에 유지됨

[Reduction 활성화 시 - Compaction]
  메시지 수: 85 → 60
  추정 토큰: 2,972 → 2,350
  절약된 토큰: 622 (20.9%)
  → 오래된 도구 호출/결과가 제거되어 컨텍스트 효율화


### 실험 3: 전략 조합 효과 시뮬레이션

모든 전략을 함께 적용했을 때의 시너지 효과

In [15]:
print("=" * 60)
print("시나리오: 복잡한 연구 작업 수행")
print("=" * 60)
print()

scenario = {
    "대화 턴 수": 50,
    "도구 호출 수": 40,
    "대용량 결과 수": 5,
    "평균 결과 크기": "100k 자",
}

print("[시나리오 설정]")
for k, v in scenario.items():
    print(f"  {k}: {v}")
print()

baseline_context = 50 * 500 + 40 * 300 + 5 * 100000
print("[모든 전략 비활성화 시]")
print(f"  예상 컨텍스트 크기: {baseline_context:,} 자 (~{baseline_context//4:,} 토큰)")
print(f"  문제: 컨텍스트 윈도우 초과 가능성 높음")
print()

with_offloading = 50 * 500 + 40 * 300 + 5 * 1000
print("[Offloading만 활성화 시]")
print(f"  예상 컨텍스트 크기: {with_offloading:,} 자 (~{with_offloading//4:,} 토큰)")
print(f"  절약: {(baseline_context - with_offloading):,} 자 ({(baseline_context - with_offloading)/baseline_context*100:.1f}%)")
print()

with_reduction = with_offloading * 0.6
print("[Offloading + Reduction 활성화 시]")
print(f"  예상 컨텍스트 크기: {int(with_reduction):,} 자 (~{int(with_reduction)//4:,} 토큰)")
print(f"  총 절약: {int(baseline_context - with_reduction):,} 자 ({(baseline_context - with_reduction)/baseline_context*100:.1f}%)")
print()

print("[+ Caching 활성화 시 추가 효과]")
print(f"  시스템 프롬프트 캐싱으로 반복 호출 비용 90% 절감")
print(f"  응답 속도 향상")

시나리오: 복잡한 연구 작업 수행

[시나리오 설정]
  대화 턴 수: 50
  도구 호출 수: 40
  대용량 결과 수: 5
  평균 결과 크기: 100k 자

[모든 전략 비활성화 시]
  예상 컨텍스트 크기: 537,000 자 (~134,250 토큰)
  문제: 컨텍스트 윈도우 초과 가능성 높음

[Offloading만 활성화 시]
  예상 컨텍스트 크기: 42,000 자 (~10,500 토큰)
  절약: 495,000 자 (92.2%)

[Offloading + Reduction 활성화 시]
  예상 컨텍스트 크기: 25,200 자 (~6,300 토큰)
  총 절약: 511,800 자 (95.3%)

[+ Caching 활성화 시 추가 효과]
  시스템 프롬프트 캐싱으로 반복 호출 비용 90% 절감
  응답 속도 향상


### 실험 4: 실제 에이전트 실행 비교

실제 에이전트를 다른 설정으로 생성하여 비교합니다.

In [17]:
from context_engineering_research_agent import create_context_aware_agent

print("에이전트 생성 비교")
print("=" * 60)

configs = [
    {"name": "기본 (모두 비활성화)", "offloading": False, "reduction": False, "caching": False},
    {"name": "Offloading만", "offloading": True, "reduction": False, "caching": False},
    {"name": "Reduction만", "offloading": False, "reduction": True, "caching": False},
    {"name": "모두 활성화", "offloading": True, "reduction": True, "caching": True},
]

for cfg in configs:
    agent = create_context_aware_agent(
        model="gpt-4.1",
        enable_offloading=cfg["offloading"],
        enable_reduction=cfg["reduction"],
        enable_caching=cfg["caching"],
    )
    print(f"\n[{cfg['name']}]")
    print(f"  Offloading: {'✅' if cfg['offloading'] else '❌'}")
    print(f"  Reduction:  {'✅' if cfg['reduction'] else '❌'}")
    print(f"  Caching:    {'✅' if cfg['caching'] else '❌'}")
    print(f"  에이전트 타입: {type(agent).__name__}")

print("\n" + "=" * 60)
print("모든 에이전트가 성공적으로 생성되었습니다.")

에이전트 생성 비교

[기본 (모두 비활성화)]
  Offloading: ❌
  Reduction:  ❌
  Caching:    ❌
  에이전트 타입: CompiledStateGraph

[Offloading만]
  Offloading: ✅
  Reduction:  ❌
  Caching:    ❌
  에이전트 타입: CompiledStateGraph

[Reduction만]
  Offloading: ❌
  Reduction:  ✅
  Caching:    ❌
  에이전트 타입: CompiledStateGraph

[모두 활성화]
  Offloading: ✅
  Reduction:  ✅
  Caching:    ✅
  에이전트 타입: CompiledStateGraph

모든 에이전트가 성공적으로 생성되었습니다.


#### (실행) ToolCallLimitMiddleware로 반복 행동(Distraction) 억제

- Baseline: 같은 `web_search`를 반복 호출
- With `ToolCallLimitMiddleware(tool_name='web_search', run_limit=1)`: 2회차부터 차단되어 다른 행동으로 전환


In [19]:
from __future__ import annotations


@tool(description="Dummy web search tool")
def web_search(query: str) -> str:
    return f"(dummy) result for {query!r}"


@tool(description="Write a todo list")
def write_todos(todos: list[str]) -> str:
    return json.dumps({"todos": todos})


class LoopingSearchModel(BaseChatModel):
    def bind_tools(self, tools: list[Any], **kwargs: Any):  # noqa: ANN401
        _ = kwargs
        self._tool_names = [t.name for t in tools if hasattr(t, 'name')]
        return self

    @property
    def _llm_type(self) -> str:
        return 'looping-search'

    @property
    def _identifying_params(self) -> dict[str, Any]:
        return {}

    def _generate(self, messages: list[BaseMessage], stop=None, run_manager=None, **kwargs: Any) -> ChatResult:
        _ = (stop, run_manager, kwargs)

        # Count tool outcomes (robust stop conditions).
        ok_search_results = [
            m
            for m in messages
            if isinstance(m, ToolMessage) and m.name == 'web_search' and (m.status is None or m.status == 'success')
        ]
        error_search_results = [
            m
            for m in messages
            if isinstance(m, ToolMessage) and m.name == 'web_search' and m.status == 'error'
        ]
        has_todo_result = any(isinstance(m, ToolMessage) and m.name == 'write_todos' for m in messages)

        # If we already wrote a todo list, end the run (avoid infinite tool-call loops).
        if has_todo_result:
            return ChatResult(generations=[ChatGeneration(message=AIMessage(content='FINAL todo list written'))])

        if len(ok_search_results) < 3 and not error_search_results:
            tcid = f"call_{uuid.uuid4().hex[:8]}"
            msg = AIMessage(
                content=f"search loop {len(ok_search_results)+1}",
                tool_calls=[{'id': tcid, 'name': 'web_search', 'args': {'query': 'context engineering'}, 'type': 'tool_call'}],
            )
            return ChatResult(generations=[ChatGeneration(message=msg)])

        # If blocked/error occurred (or we reached 3 searches), switch to planning once.
        tcid = f"call_{uuid.uuid4().hex[:8]}"
        msg = AIMessage(
            content='switch to todos',
            tool_calls=[{'id': tcid, 'name': 'write_todos', 'args': {'todos': ['summarize findings']}, 'type': 'tool_call'}],
        )
        return ChatResult(generations=[ChatGeneration(message=msg)])


user = HumanMessage(content="Context engineering을 조사해줘")
state = {"messages": [user]}

print("=" * 60)
print("[Baseline] 제한 없음")
print("=" * 60)
agent_baseline = create_agent(model=LoopingSearchModel(), tools=[web_search, write_todos], middleware=[])
res1 = agent_baseline.invoke(state, {"configurable": {"thread_id": "exp7_baseline"}})
_print_messages(res1["messages"])

print("\n" + "=" * 60)
print("[With ToolCallLimitMiddleware] web_search run_limit=1")
print("=" * 60)
limiter = ToolCallLimitMiddleware(tool_name='web_search', run_limit=1, exit_behavior='continue')
agent_limited = create_agent(model=LoopingSearchModel(), tools=[web_search, write_todos], middleware=[limiter])
res2 = agent_limited.invoke(state, {"configurable": {"thread_id": "exp7_limited"}})
_print_messages(res2["messages"])


[Baseline] 제한 없음
00 HUMAN: Context engineering을 조사해줘
01 AI: search loop 1
     tool_call: name=web_search id=call_76fe1482 args={'query': 'context engineering'}
02 TOOL: name=web_search status=success id=call_76fe1482
     content: (dummy) result for 'context engineering'
03 AI: search loop 2
     tool_call: name=web_search id=call_9e523f0a args={'query': 'context engineering'}
04 TOOL: name=web_search status=success id=call_9e523f0a
     content: (dummy) result for 'context engineering'
05 AI: search loop 3
     tool_call: name=web_search id=call_ea484b66 args={'query': 'context engineering'}
06 TOOL: name=web_search status=success id=call_ea484b66
     content: (dummy) result for 'context engineering'
07 AI: switch to todos
     tool_call: name=write_todos id=call_61a16615 args={'todos': ['summarize findings']}
08 TOOL: name=write_todos status=success id=call_61a16615
     content: {"todos": ["summarize findings"]}
09 AI: FINAL todo list written

[With ToolCallLimitMiddleware] web_se

### 실험 8: Context Poisoning (검증되지 않은 사실의 오염)

검증되지 않은 정보가 컨텍스트/메모리에 들어가면, 이후 의사결정이 그 “오염된 사실”을 기반으로 굳어질 수 있습니다.

이 실험은:

- 출처 없는 메모리 항목(검증되지 않음)이 이후 판단에 끼어드는 상황
- 완화책: **출처 태깅 + 검증 게이트(verified only)**

을 시뮬레이션합니다.


In [20]:
from __future__ import annotations

from dataclasses import dataclass


@dataclass(frozen=True)
class MemoryItem:
    key: str
    value: str
    source: str | None  # tool_call_id 등
    verified: bool


def choose_plan(memory: list[MemoryItem]) -> str:
    # Toy planner: 메모리를 그대로 신뢰한다(나쁜 예)
    installed = next((m.value for m in memory if m.key == "package_installed"), "unknown")
    if installed == "yes":
        return "Skip install; proceed to use the package."
    if installed == "no":
        return "Install the package."
    return "Check whether the package is installed."


def choose_plan_verified_only(memory: list[MemoryItem]) -> str:
    verified = [m for m in memory if m.verified]
    return choose_plan(verified)


memory_clean = [
    MemoryItem(key="package_installed", value="no", source="tool_call_1", verified=True),
]

memory_poisoned = [
    MemoryItem(key="package_installed", value="no", source="tool_call_1", verified=True),
    # 오염: 출처 없음 + 검증되지 않음
    MemoryItem(key="package_installed", value="yes", source=None, verified=False),
]

print("=" * 60)
print("[A] 정상 메모리")
print("=" * 60)
print("blind plan:", choose_plan(memory_clean))
print("verified-only plan:", choose_plan_verified_only(memory_clean))
print()

print("=" * 60)
print("[B] 오염된 메모리(Poisoning)")
print("=" * 60)
print("blind plan:", choose_plan(memory_poisoned))
print("verified-only plan:", choose_plan_verified_only(memory_poisoned))
print()

print("=" * 60)
print("[C] 완화책: 출처 없는 사실은 검증 요청으로 라우팅")
print("=" * 60)
needs_verification = [m for m in memory_poisoned if (m.source is None or not m.verified)]
print("needs_verification:")
for item in needs_verification:
    print(f"  - {item.key}='{item.value}' source={item.source} verified={item.verified}")
print("\n→ 정책: tool로 재확인 후에만 state/memory에 반영")


[A] 정상 메모리
blind plan: Install the package.
verified-only plan: Install the package.

[B] 오염된 메모리(Poisoning)
blind plan: Install the package.
verified-only plan: Install the package.

[C] 완화책: 출처 없는 사실은 검증 요청으로 라우팅
needs_verification:
  - package_installed='yes' source=None verified=False

→ 정책: tool로 재확인 후에만 state/memory에 반영


#### (실행) 검증 게이트(Verification Gate)로 Poisoning 차단

- Baseline: `verified=false` 결과를 그대로 믿고 잘못된 계획을 수립
- With gate middleware: `verified=false` 사실은 차단하고, 검증된 tool로 재확인하도록 강제


In [21]:
from __future__ import annotations

from langchain.agents.middleware.types import AgentState
from langgraph.runtime import Runtime


@tool(description="Unverified guess of install status")
def guess_install_status() -> str:
    # Poisoned / unverified
    return json.dumps({"package_installed": "yes", "verified": False, "source": "guess"})


@tool(description="Verified scan of install status")
def scan_install_status() -> str:
    # Verified
    return json.dumps({"package_installed": "no", "verified": True, "source": "scan"})


class VerificationGateMiddleware(AgentMiddleware):
    def before_model(self, state: AgentState, runtime: Runtime[Any]) -> dict[str, Any] | None:  # noqa: ARG002
        messages = state.get('messages', [])

        # Avoid repeatedly injecting the same constraint.
        if any(isinstance(m, SystemMessage) and 'UNVERIFIED_FACT_BLOCKED' in m.content for m in messages):
            return None
        if any(isinstance(m, ToolMessage) and m.name == 'scan_install_status' for m in messages):
            return None

        # If we see an unverified tool result, inject a system constraint.
        for m in reversed(messages):
            if isinstance(m, ToolMessage) and m.name == 'guess_install_status':
                try:
                    data = json.loads(str(m.content))
                except json.JSONDecodeError:
                    continue
                if data.get('verified') is False:
                    patched = list(messages)
                    patched.append(
                        SystemMessage(
                            content=(
                                'UNVERIFIED_FACT_BLOCKED: Do not trust guess_install_status. '
                                'Call scan_install_status and decide based on verified=true only.'
                            )
                        )
                    )
                    return {'messages': Overwrite(patched)}
        return None


class InstallPlannerModel(BaseChatModel):
    def bind_tools(self, tools: list[Any], **kwargs: Any):  # noqa: ANN401
        _ = kwargs
        self._tool_names = [t.name for t in tools if hasattr(t, 'name')]
        return self

    @property
    def _llm_type(self) -> str:
        return 'install-planner'

    @property
    def _identifying_params(self) -> dict[str, Any]:
        return {}

    def _generate(self, messages: list[BaseMessage], stop=None, run_manager=None, **kwargs: Any) -> ChatResult:
        _ = (stop, run_manager, kwargs)

        # If scan result exists, finalize.
        for m in reversed(messages):
            if isinstance(m, ToolMessage) and m.name == 'scan_install_status':
                data = json.loads(str(m.content))
                decision = 'INSTALL' if data.get('package_installed') == 'no' else 'SKIP'
                return ChatResult(generations=[ChatGeneration(message=AIMessage(content=f"FINAL decision={decision} (source=scan)"))])

        blocked = any(
            isinstance(m, SystemMessage) and 'UNVERIFIED_FACT_BLOCKED' in m.content for m in messages
        )

        if blocked:
            tcid = f"call_{uuid.uuid4().hex[:8]}"
            msg = AIMessage(content='scan', tool_calls=[{'id': tcid, 'name': 'scan_install_status', 'args': {}, 'type': 'tool_call'}])
            return ChatResult(generations=[ChatGeneration(message=msg)])

        # Baseline behavior: trust guess first.
        if not any(isinstance(m, ToolMessage) and m.name == 'guess_install_status' for m in messages):
            tcid = f"call_{uuid.uuid4().hex[:8]}"
            msg = AIMessage(content='guess', tool_calls=[{'id': tcid, 'name': 'guess_install_status', 'args': {}, 'type': 'tool_call'}])
            return ChatResult(generations=[ChatGeneration(message=msg)])

        # If guess exists and no gate, finalize (poisoned).
        for m in reversed(messages):
            if isinstance(m, ToolMessage) and m.name == 'guess_install_status':
                data = json.loads(str(m.content))
                decision = 'SKIP' if data.get('package_installed') == 'yes' else 'INSTALL'
                return ChatResult(generations=[ChatGeneration(message=AIMessage(content=f"FINAL decision={decision} (source=guess)"))])

        return ChatResult(generations=[ChatGeneration(message=AIMessage(content='FINAL no decision'))])


user = HumanMessage(content='패키지 X 설치가 필요한지 판단해줘')
state = {"messages": [user]}

tools = [guess_install_status, scan_install_status]

print("=" * 60)
print("[Baseline] verification gate 없음")
print("=" * 60)
agent_baseline = create_agent(model=InstallPlannerModel(), tools=tools, middleware=[])
res1 = agent_baseline.invoke(state, {"configurable": {"thread_id": "exp8_baseline"}})
_print_messages(res1['messages'])

print("\n" + "=" * 60)
print("[With VerificationGateMiddleware]")
print("=" * 60)
agent_gated = create_agent(model=InstallPlannerModel(), tools=tools, middleware=[VerificationGateMiddleware()])
res2 = agent_gated.invoke(state, {"configurable": {"thread_id": "exp8_gated"}})
_print_messages(res2['messages'])


[Baseline] verification gate 없음
00 HUMAN: 패키지 X 설치가 필요한지 판단해줘
01 AI: guess
     tool_call: name=guess_install_status id=call_ccef8e23 args={}
02 TOOL: name=guess_install_status status=success id=call_ccef8e23
     content: {"package_installed": "yes", "verified": false, "source": "guess"}
03 AI: FINAL decision=SKIP (source=guess)

[With VerificationGateMiddleware]
00 HUMAN: 패키지 X 설치가 필요한지 판단해줘
01 AI: guess
     tool_call: name=guess_install_status id=call_70cc8071 args={}
02 TOOL: name=guess_install_status status=success id=call_70cc8071
     content: {"package_installed": "yes", "verified": false, "source": "guess"}
03 SYSTEM: UNVERIFIED_FACT_BLOCKED: Do not trust guess_install_status. Call scan_install_status and decide based on verified=true only.
04 AI: scan
     tool_call: name=scan_install_status id=call_9e23662c args={}
05 TOOL: name=scan_install_status status=success id=call_9e23662c
     content: {"package_installed": "no", "verified": true, "source": "scan"}
06 AI: FINAL deci

---

## 요약

### Context Engineering 5가지 전략 요약

| 전략 | 트리거 조건 | 효과 |
|------|------------|------|
| **Offloading** | 20k 토큰 초과 | 파일로 축출 |
| **Reduction** | 85% 사용량 초과 | Compaction/Summarization |
| **Retrieval** | 파일 접근 필요 | grep/glob 검색 |
| **Isolation** | 복잡한 작업 | SubAgent 위임 |
| **Caching** | 1k+ 토큰 시스템 프롬프트 | Prompt Caching |

### 핵심 인사이트

1. **파일시스템 = 외부 메모리**: 컨텍스트 윈도우는 제한되어 있지만, 파일시스템은 무한
2. **점진적 공개**: 모든 정보를 한 번에 로드하지 않고 필요할 때만 로드
3. **격리된 실행**: SubAgent로 컨텍스트 오염 방지
4. **자동화된 관리**: 에이전트가 직접 컨텍스트를 관리하도록 미들웨어 설계
5. **실패 모드 방어**: Poisoning/Distraction/Confusion/Clash를 관측하고 완화하는 규칙이 필요
6. **도구도 컨텍스트다**: 도구 설명/목록도 최소화하고, 필요할 때만 로딩(또는 계층화)

### 추가 실험(실패 모드)

- **Confusion**: 유사 도구가 많을수록 선택이 불안정해짐(도구 로딩 제한/계층화로 완화)
- **Clash**: 모순 관찰을 충돌로 기록하고 재검증으로 해소
- **Distraction**: 장기 로그에서 반복 행동 쏠림(계획/다음 행동 강제로 완화)
- **Poisoning**: 출처 없는 사실을 차단하고 검증 게이트로 통제

### 추가 실험(실패 모드) - 실제 실행 기반

- **Tool Selection**: `LLMToolSelectorMiddleware`로 tool set을 축소해 Confusion 완화
- **Tool Call Limiting**: `ToolCallLimitMiddleware`로 반복 tool call을 차단해 Distraction 완화
- **Filesystem Tools**: deepagents `FilesystemMiddleware`로 `ls/read_file/glob/grep` 실제 실행 로그 확인
- **Custom Guards**: (실험 목적) 충돌 감지/검증 게이트를 `AgentMiddleware`로 구현해 Clash/Poisoning 완화
